In [359]:
#####Robust Network Co-Clustering#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import gensim
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

In [360]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [361]:
####データの発生####
##データの設定
k1 = 15   #ユーザーのクラスタ数
k21 = 12   #アイテムの関係オブジェクトのクラスタ数
k22 = 8   #アイテムの無関係オブジェクトのクラスタ数
hh = 10000   #ユーザー数
item = 5000   #アイテム数
pt = np.random.poisson(np.random.gamma(25, 1/0.2, hh))   #ユーザーあたりのレコード数
f = np.sum(pt)   #総レコード数

In [362]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.array(range(f))
user_index = [i for i in range(hh)]
user_vec = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]
    user_vec[i] = np.repeat(1, len(user_index[i]))

In [363]:
##アイテムの割当を生成
#セグメント割当を生成
topic = 25
phi = np.random.dirichlet(np.repeat(0.5, item), topic)
theta = np.random.dirichlet(np.repeat(2.5, topic), hh)
z = np.dot(np.array([np.random.multinomial(1, theta[i, :], 1) for i in range(hh)]).reshape(hh, topic), range(topic))

#多項分布からアイテムを生成
item_id = np.zeros(f, dtype='int')
for i in range(hh):
    if i%1000==0:
        print(i)
    item_id[user_index[i]] = np.dot(np.random.multinomial(1, phi[z[i], :], pt[i]), range(item))
    
#インデックスの設定
item_index = [j for j in range(item)]
item_vec = [j for j in range(item)]
for j in range(item):
    item_index[j] = index[item_id==j]
    item_vec[j] = np.repeat(1, len(item_index[j]))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [364]:
##セグメント割当を生成
#関係有無の潜在変数rを生成
tau = 0.4; taut = tau
r = np.random.binomial(1, tau, item); rt = r
index_r0 = np.array(range(item))[r==0]; n0 = index_r0.shape[0]
index_r1 = np.array(range(item))[r==1]; n1 = index_r1.shape[0]

#ユーザーセグメントを生成
alpha1 = np.random.dirichlet(np.repeat(5.0, k1), 1).reshape(-1); alphat1 = alpha1
Z1 = np.random.multinomial(1, alpha1, hh)
z_vec1 = np.dot(Z1, np.arange(k1))

#アイテムセグメントを生成
alpha21 = np.random.dirichlet(np.repeat(5.0, k21), 1).reshape(-1); alphat21 = alpha21
alpha22 = np.random.dirichlet(np.repeat(5.0, k22), 1).reshape(-1); alphat22 = alpha22
Z21 = np.zeros((hh, k21)); Z22 = np.zeros((hh, k22))
Z21[index_r1, ] = np.random.multinomial(1, alpha21, n1)
Z22[index_r0, ] = np.random.multinomial(1, alpha22, n0)
z_vec21 = np.array(np.dot(Z21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Z22, np.arange(k22)), dtype="int")

##ベータ分布からセグメントごとの関係確率を生成
#関係オブジェクトのパラメータを生成
alpha1 = 0.2; beta1 = 0.5
theta = np.random.beta(alpha1, beta1, k1*k21).reshape(k1, k21)
thetat = theta

#無関係オブジェクトのパラメータを生成
alpha2 = 0.6; beta2 = 1.0
phi = np.random.beta(alpha2, beta2, k22)
phit = phi

##関係データを生成
#セグメント割当から関係確率を設定
r_vec = r[item_id]
Prob = r_vec * np.sum(theta[z_vec1[user_id], ] * Z21[item_id, ], axis=1) + (1-r_vec) * phi[z_vec22[item_id]]

#ベルヌーイ分布から関係データを生成
y = np.random.binomial(1, Prob, f)
y_vec = y.reshape(f, 1)
index_y0 = np.array(range(f))[y==0]
index_y1 = np.array(range(f))[y==1]
n0 = np.sum(1-y)
n1 = np.sum(y)

In [365]:
####ギブスサンプリングでRobust Network Co-Clusteringをサンプリング#####
##アルゴリズムの設定
LL1 = -100000000   #対数尤度の初期値
R = 2000
keep = 2  
iter = 0
burnin = 500/keep
disp = 10

In [366]:
##事前分布の設定
#混合率の事前分布
s0 = 0.1; v0 = 0.2
alpha1 = 0.25
alpha2 = 0.25

#モデルパラメータの事前分布
beta11 = 0.5; beta12 = 0.5
beta21 = 0.5; beta22 = 0.5

In [389]:
##パラメータの真値
#混合率の真値
tau = taut
alpha1 = alphat1
alpha21 = alphat21
alpha22 = alphat22

#モデルパラメータの真値
theta = thetat
phi = phit

#潜在変数の真値
r = rt 
Zi1 = Z1
Zi21 = Z21
Zi22 = Z22
z_vec1 = np.dot(Zi1, np.arange(k1))
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

In [390]:
####ギブスサンプリングでパラメータをサンプリング####

##潜在変数rをサンプリング
#無関係オブジェクトのクラスタごとの対数尤度
phi_dt = np.zeros((f, k22))
phi_dt[index_y1, ] = np.log(np.repeat(phi, n1).reshape(n1, k22, order="F"))
phi_dt[index_y0, ] = np.log(np.repeat(1-phi, n0).reshape(n0, k22, order="F"))

#関係オブジェクトのクラスタごとの対数尤度
theta_dt = np.zeros((f, k21))
theta_dt[index_y1, ] = np.log(theta[z_vec1[user_id[index_y1]], ])
theta_dt[index_y0, ] = np.log(1 - theta[z_vec1[user_id[index_y0]], ])

#クラスタごとの対数尤度の和
LLi0 = np.zeros((item, k22)); LLi1 = np.zeros((item, k21))
for j in range(item):
    LLi0[j] = np.dot(item_vec[j], phi_dt[item_index[j], ])
    LLi1[j] = np.dot(item_vec[j], theta_dt[item_index[j], ])
    
#潜在変数rの割当確率
LLi_max = np.max(np.hstack((LLi0, LLi1)), axis=1).reshape(item, 1)
Li0 = np.sum(np.repeat(alpha22, item).reshape(item, k22) * np.exp(LLi0 - LLi_max), axis=1)
Li1 = np.sum(np.repeat(alpha21, item).reshape(item, k21) * np.exp(LLi1 - LLi_max), axis=1)
Prob = (tau*Li1) / ((1-tau)*Li0 + tau*Li1)   

r = np.random.binomial(1, Prob, item)
index_r1 = np.arange(item)[r==1]; rn1 = index_r1.shape[0]
index_r0 = np.delete(np.arange(item), index_r); rn0 = index_r0.shape[0]

In [391]:
##アイテムのクラスタをサンプリング
#無関係オブジェクトのクラスタをサンプリング
Zi22 = np.zeros((item, k22))
LLi_max = np.max(LLi0[index_r0, ], axis=1).reshape(rn0, 1)
Li = np.repeat(alpha22, rn0).reshape(rn0, k22, order="F") * np.exp(LLi0[index_r0, ] - LLi_max)
Prob = Li / np.sum(Li, axis=1).reshape(rn0, 1)
Zi22[index_r0, ] = np.array(rmnom(Prob, rn0, k22, np.arange(rn0)).todense())
z_vec22 = np.array(np.dot(Zi22, np.arange(k22)), dtype="int")

#関係オブジェクトのクラスタをサンプリング
Zi21 = np.zeros((item, k21))
LLi_max = np.max(LLi1[index_r1, ], axis=1).reshape(rn1, 1)
Li = np.repeat(alpha21, rn1).reshape(rn1, k21, order="F") * np.exp(LLi1[index_r1, ] - LLi_max)
Prob = Li / np.sum(Li, axis=1).reshape(rn1, 1)
Zi21[index_r1, ] = np.array(rmnom(Prob, rn1, k21, np.arange(rn1)).todense())
z_vec21 = np.array(np.dot(Zi21, np.arange(k21)), dtype="int")

In [408]:
##ユーザーのクラスタをサンプリング
#アイテムのクラスタの条件付き尤度を設定
theta_T = theta.T
LLi = np.zeros((hh, k1)); Lho = np.zeros((f, k1))
Lho[index_y1, ] = np.log(theta_T[z_vec21[item_id[index_y1]], ]) *  r[item_id[index_y1]].reshape(n1, 1) 
Lho[index_y0, ] = np.log(1 - theta_T[z_vec21[item_id[index_y0]], ]) * r[item_id[index_y0]].reshape(n0, 1)
for i in range(hh):
    LLi[i, ] = np.dot(user_vec[i], Lho[user_index[i], ])
    
#ベルヌーイ分布からクラスタをサンプリング
Li = np.repeat(alpha1, hh).reshape(hh, k1, order="F") * np.exp(LLi - np.max(LLi, axis=1).reshape(hh, 1))
Prob = Li / np.sum(Li, axis=1).reshape(hh, 1)   #クラスタの割当確率
Zi1 = np.array(rmnom(Prob, hh, k1, np.arange(hh)).todense())
z_vec1 = np.array(np.dot(Zi1, np.arange(k1)), dtype="int")

In [412]:
y

array([0, 0, 0, ..., 0, 0, 1])

In [413]:
Zi1

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])